# 🔧 에이전트를 MCP 도구로 노출하기

> **개요**  
> 모델 컨텍스트 프로토콜(MCP)을 사용해서 에이전트를 도구로 노출하여 MCP 도구를 지원하는 다른 시스템에서 활용하는 방법을 알아봅니다.

---

## 📋 에이전트를 MCP 서버로 노출

`as_mcp_server()` 메서드를 사용하면 에이전트를 MCP 서버로 변환할 수 있습니다. 이를 통해 MCP 호환 클라이언트에서 해당 에이전트를 도구로 호출할 수 있습니다.

먼저 MCP 서버로 노출할 에이전트를 생성합니다. 필요한 경우 Function 도구를 추가할 수도 있습니다.

In [ ]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 오늘의 스페셜 메뉴를 반환하는 도구
def get_specials() -> Annotated[str, "메뉴에서 스페셜 메뉴를 반환합니다. 🍽️✨"]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

# 메뉴 아이템의 가격을 반환하는 도구. 9.99 달러로 고정
def get_item_price(
    menu_item: Annotated[str, "메뉴 아이템의 이름입니다. 🍴"],
) -> Annotated[str, "메뉴 아이템의 가격을 반환합니다. 💰"]:
    return "$9.99"

# RestaurantAgent 에이전트 생성 및 도구 제공
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="RestaurantAgent",
    instructions="메뉴에 대한 질문에 답변합니다. 🍽️💬",
    tools=[get_specials, get_item_price],
)

---

## 🔄 에이전트를 MCP 서버로 전환

에이전트의 이름과 설명은 MCP 서버 메타데이터로 사용됩니다.

In [ ]:
# Expose the agent as an MCP server
server = agent.as_mcp_server()

---

## ▶️ MCP 서버 실행

아래 코드를 실행하여 MCP 서버를 시작합니다. 실행 후 MCP 호환 클라이언트에서 이 에이전트를 도구로 호출할 수 있습니다.

```python
# 에이전트를 MCP 서버로 전환
server = agent.as_mcp_server()

async def run():
    async with stdio_server() as (read_stream, write_stream):
        await server.run(read_stream, write_stream, server.create_initialization_options())

if __name__ == "__main__":
    anyio.run(run)
```

> **💡 참고**  
> 노트북 환경에서는 실행에 제약이 있어, 전체 코드를 별도의 Python 파일로 저장하여 실행합니다.  
> 완성된 파일은 현재 폴더의 `restaurant_mcp_server.py`에서 확인할 수 있습니다.

---

## 🧪 MCP 서버 테스트 (선택사항)

### 📌 서버 실행

별도의 터미널에서 다음 명령어로 MCP 서버를 실행하세요:

```bash
python MAF/restaurant_mcp_server.py
```

### 🔗 클라이언트 연결 (선택사항)

MCP 클라이언트(예: Claude Desktop)에서 이 서버에 연결하여 `RestaurantAgent`를 도구로 사용할 수 있습니다.

> **📖 자세한 테스트 방법**  
> 아래 링크에서 Claude Desktop을 사용한 테스트 방법을 확인할 수 있습니다.
>
> [Claude Desktop에서 서버 테스트하기](https://github.com/warnov/ms-agent-framework-step-by-step-workshop/blob/main/07-agent-as-MCP-tool/README.md#:~:text=Trying%20the%20server%20from%20Claude%20Desktop)

### 💡 추가 참고 사항

#### Azure AI Agent와 로컬 🛠️Model Context Protocol (MCP) 서버의 통합

로컬에서 Microsoft Learn MCP 도구를 에이전트와 통합하여 활용하려면 다음의 코드를 참고하세요:

```python

In [ ]:
import asyncio

from agent_framework import MCPStreamableHTTPTool
from agent_framework.azure import AzureAIProjectAgentProvider
from azure.identity.aio import AzureCliCredential

"""
이 실습은 Azure AI Agent와 로컬 Model Context Protocol (MCP) 서버의 통합을 보여줍니다.
"""

async def main() -> None:
    """AzureAIProjectAgentProvider와 함께 로컬 MCP 도구 사용 예제."""
    print("=== Azure AI Agent with Local MCP Tools Example ===\n")

    # Microsoft Learn MCP 도구 설정
    mcp_tool = MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
    )

    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="DocsAgent",
            instructions="당신은 Mircosoft Learn MCP를 사용하여,기술문서 질문에 답변을 해주는 친절한 에이전트입니다.",
            tools=mcp_tool,
        )

        # Use agent as context manager to ensure proper cleanup
        async with agent:
            # First query
            first_query = "az cli를 사용해서 Azure storage account를 만드는 방법은?"
            print(f"User: {first_query}")
            first_result = await agent.run(first_query)
            print(f"Agent: {first_result}")
            print("\n=======================================\n")
            # Second query
            second_query = "Microsoft Agent Framework란 무엇인가요?"
            print(f"User: {second_query}")
            second_result = await agent.run(second_query)
            print(f"Agent: {second_result}")


if __name__ == "__main__":
    await main()